# 2.2 Ітераційні методи розв'язування СЛАР
---------------------

## 2.2.4.  Метод Зейделя

Нехай маємо СЛАР

$(1)\qquad\qquad\qquad A\,x = b,$

де  $x \in \mathbb{R}^{n}$  --- невідомий вектор, $n \ge 2$ - довільне натуральне число,
$A\in \mathbb{R}^{n\times n},\; b\in \mathbb{R}^{n}$ - задані квадратна матриця і вектор вільних членів відповідно:

$(2)\qquad\qquad
A=
\begin{pmatrix}
a_{11} & a_{12} &\ldots & a_{1n} \\
a_{21} & a_{22} &\ldots & a_{2n} \\
\ldots & \ldots & \ldots \\
a_{n1} & a_{n2} &\ldots & a_{nn}
\end{pmatrix},\quad  
b=
\begin{pmatrix}
b_{1}\\
b_{2}\\
\ldots  \\
b_{n}
\end{pmatrix}.
$

Вважаємо, що $\mathrm{det} A\neq 0$, а також $a_{ii} \neq 0, \; i \in \overline {1, n}$.


Метод *Зейделя* полягає в обчисленні послідовності 

$\qquad\qquad x^0, x^1, \ldots, x^k, \ldots$

наближень розв'язку системи за правилом:

$(3)\qquad\qquad\qquad
x_{i}^{k+1} =  - \sum_{j=1}^{i-1} \frac{a_{ij}}{a_{ii}} x_j^{k+1} - \sum_{j=i+1}^n
\frac{a_{ij}}{a_{ii}} x_j^k + d_i,\quad
 i =\overline{1,n},\ \ \ k\in \mathbb{N}_0.
$

де 
$x^k =(x_1^k,x_2^k, \dots , x_n^k)^\top$ --- $k$-та ітерація вектора наближень, $x^0 \in \mathbb{R}^n$ --- початкове наближення.


#### Пояснення до використання програмного коду
-----------------

*   Підготувати  функції, які потрібні для реалізації методу: 
    1.   виконати комірку з імпортом ``NumPy``
    2.   виконати комірку з визначеннями допоміжних функцій ``norm_1``,``norm_2`` i ``norm_3``    
    3.   виконати комірку з визначеннями функції ``simple_iteration`` 
    4.   виконати комірку з визначеннями функцій ``Jacobi_modification`` і ``Seidel_solver``
*   Підготувати  функції, які задають конкретну СЛАР і дані чисельного методу
    1.   виконати комірки з визначеннями функцій ``set_matrix`` i ``set_vector`` 
    2.   виконати комірку з визначеннями функцій ``set_x0`` 
    3.   виконати комірку із заданням точності ``eps`` і максимальної кількості ітерацій ``max_iter``
*   Виконати комірку з викликом функції ``Seidel_solver``

In [2]:
import numpy as np

In [3]:
# Допоміжні функції

def norm_1(a):
    """обчислення норми_1 матриці a"""   
    m=0
    for i in range(a.shape[0]):
        s=0
        for j in range(a.shape[1]):
            s+=abs(a[i][j])
        if s>m:
            m=s
    return m

def norm_2(a):
    """обчислення норми_2 матриці a"""   
    m=0
    for j in range(a.shape[1]):
        s=0
        for i in range(a.shape[0]):
            s+=abs(a[i][j])
        if s>m:
            m=s
    return m
def normv_1(a):
    for i in range(len(a)):
        a[i]=np.abs(a[i])
    return np.max(a)

def norm_3(a):
    """обчислення евклідової норми вектора a"""
    return np.sqrt(np.sum(a**2))

>#### ``Jacobi_modification`` модифікує матрицю і вектор вільних членів СЛАР (1)     

-------------------

In [4]:
def Jacobi_modification(a, b):
    """ модифікація матриці і вектора вільних членів СЛАР """
    for i in range(a.shape[0]):
        b[i] /= a[i,i]
        a[i,:] /= -a[i,i]
        a[i,i]=0
        

>### Функція  ``Seidel_iteration``  реалізує метод Зейделя для розв'язування СЛАР

-------------------
*   На кожній ітерації $k$ для зберігання попереднього наближення розв'язку $x^{k-1}$ використовуємо вектор x_prev, 
*   Після обчислення нового значення $x^{k}$ зберігаємо його у векторі x_new  
    



In [5]:
def Seidel_iteration(b,d,x0,eps,iter_print):
    """ послідовно обчислює наближення розв'язку СЛАР 
      за методом Зейделя, поки евклідова норма 
      різниці двох послідовних наближень не стане меншою заданої точності eps      
      """
    n=b.shape[0]
    x_prev=x0.copy()
    
    k=1
    x_n = np.empty(n) 
    x_n[0]=np.matmul(b[0,1:],x_prev[1:]).sum() +d[0]
    for i in range(1,n):
        x_n[i]= np.matmul(b[i,:i],x_n[:i]).sum() + np.matmul(b[i,i+1:],x_prev[i+1:]).sum() +d[i] 

    while normv_1(x_n - x_prev)  > eps and k < max_iter:        
        x_prev=x_n.copy()  
        
        k+=1
        x_n[0]=np.matmul(b[0,1:],x_prev[1:]).sum() +d[0]   
        for i in range(1,n):
            x_n[i]= np.matmul(b[i,:i],x_n[:i]).sum() + np.matmul(b[i,i+1:],x_prev[i+1:]).sum() +d[i] 
            
    return k, x_n
    

>#### ``Seidel_solver`` організовує обчислення наближеного розв'язку СЛАР (1) методом Зейделя з точністю ``eps`` 
-------------------


In [6]:
def Seidel_solver(set_matrix, set_vector,set_x0, eps, max_iter):
    """ Розв'язування СЛАР методом простих ітерацій """      
    b=set_matrix()  
    d=set_vector()
    Jacobi_modification(b, d) 
    k=0
    if norm_1(b)<1 or norm_2(b)<1 : 
        x0=set_x0()        
        k, x = Seidel_iteration(b,d,x0,eps,max_iter)
        return k, x   
    else:
        return k, d 
    

>#### Обчислення наближеного розв'язку СЛАР (1) методом Зейделя з точністю ``eps``

-------------------

In [7]:
def set_matrix():
    """ функція для задання матриці СЛАР"""
    matrix=np.array([[10, 1, 2],[1,5,-1],[1,-2,10]],dtype=float )
    return matrix

In [8]:
def set_vector():
    """ функція для задання вектора вільних членів СЛАР"""
    vector=np.array([[18], [8], [27]],dtype=float)
    return vector

In [9]:
def set_x0():
    """ функція для задання вектора початкового наближення розв'язку"""
    return np.array([[1], [2], [1]],dtype=float)

In [10]:
eps=0.001
max_iter=100

In [11]:
cond, x = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if cond > 0 and cond < max_iter:
    print(f"Наближений розв'язок системи \n {x} \n обчислено за {cond} ітерацій")
elif  cond == max_iter : 
    print(f"Наближений розв'язок системи не обчислено за {cond} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Наближений розв'язок системи 
 [1.00008551 1.99994699 2.99998085] 
 обчислено за 5 ітерацій


In [12]:
from scipy import linalg

In [46]:
def Jacobi_decomposition(a,b):
    """ модифікація матриці a і вектора b вільних членів СЛАР  
        повернення M i L, M+L=a 
    """
    n=a.shape[0]
    M=np.zeros((n,n), dtype=float)
    L=np.zeros((n,n), dtype=float)
    for i in range(n):        
        b[i] /= a[i,i]
        M[i, :i+1] = a[i,:i+1] / a[i,i]
        L[i, i+1:] = a[i, i+1:] / a[i,i]
        a[i,:] /= a[i,i]
    return M,L
        

In [13]:
A=set_matrix()
A

array([[10.,  1.,  2.],
       [ 1.,  5., -1.],
       [ 1., -2., 10.]])

In [14]:
d=set_vector()
d

array([[18.],
       [ 8.],
       [27.]])

#### $A=M+L$

In [16]:
M, L = Jacobi_decomposition(A,d) 

In [17]:
d

array([[1.8],
       [1.6],
       [2.7]])

In [18]:
M

array([[ 1. ,  0. ,  0. ],
       [ 0.2,  1. ,  0. ],
       [ 0.1, -0.2,  1. ]])

In [19]:
L

array([[ 0. ,  0.1,  0.2],
       [ 0. ,  0. , -0.2],
       [ 0. ,  0. ,  0. ]])

#### $Minv = M^{-1}$

In [20]:
Minv=linalg.inv(M)
Minv

array([[ 1.  , -0.  , -0.  ],
       [-0.2 ,  1.  , -0.  ],
       [-0.14,  0.2 ,  1.  ]])

In [47]:
# перевірка
Minv.dot(M)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

#### $b = M^{-1}L$

In [48]:
b=-Minv.dot(L)
b

array([[-0.   , -0.1  , -0.2  ],
       [-0.   ,  0.02 ,  0.24 ],
       [-0.   ,  0.014,  0.068]])

In [49]:
nb1=norm_1(b) 
nb1

0.30000000000000004

In [51]:
k=int( np.log10(eps *(1-nb1) /normv_1(d1) )  / np.log10(nb1) )
k

12

In [52]:
nb2=norm_2(b)
nb2

0.508

In [53]:
k=int( np.log10(eps *(1-nb2) /normv_1(d1) )  / np.log10(nb2) )
k

22

#### $d1 = M^{-1}d$

In [26]:
d1=Minv.dot(d)
d1

array([[1.8  ],
       [1.24 ],
       [2.768]])

In [30]:
normv_1(d1)

2.7680000000000002

функції для підготовки вектора і матриці при застосуванні простої ітерації

In [31]:
def set_vector1():
    """ функція для задання вектора вільних членів СЛАР"""
    vector=np.array([[1.8  ], [1.24 ],[2.768]],dtype=float)
    return vector

In [32]:
def set_matrix1():
    """ функція для задання матриці конкретної СЛАР"""
    return np.array([[ 0.   ,  -0.1  ,  -0.2  ], [ 0.   , 0.02 , 0.24 ], [ 0.   , 0.014, 0.068]],dtype=float)

In [35]:
def set_x0():
    """ функція для задання вектора початкового наближення розв'язку"""
    return set_vector1()

In [33]:
def simple_iteration(b,d,x0,eps,max_iter):
    """ послідовно обчислює наближення розв'язку СЛАР 
      методом простої ітерації, поки евклідова норма 
      різниці двох послідовних наближень не стане меншою заданої точності eps      
      x0 -- початкове наближення """
    x_prev=x0.copy()  
    k=1
    x_new=np.matmul(b,x0)+d   
    while normv_1(x_new-x_prev) > eps and k < max_iter:
        k+=1
        x_prev=x_new
        x_new=np.matmul(b,x_prev)+d
    return k, x_new

In [34]:
def simple_iteration_solver(set_matrix, set_vector,set_x0, eps, max_iter):
    """ Розв'язування СЛАР методом простих ітерацій """      
    b=set_matrix()  
    k=0
    if norm_1(b)<1 or norm_2(b)<1 :
        d=set_vector()
        x0=set_x0()
        k, x=simple_iteration(b,d,x0,eps,max_iter)
        return k, x
    else:
        return k, b 
    

>#### Обчислення наближеного розв'язку СЛАР (1) методом Якобі-Зейделя з точністю ``eps``

-------------------

In [54]:
eps=0.000001
max_iter=100

In [55]:
cond, x=simple_iteration_solver(set_matrix1, set_vector1, set_x0, eps, max_iter)

if cond > 0 and cond < max_iter:
    print(f"Наближений розв'язок системи \n {x} \n обчислено за {cond} ітерацій")
elif  cond == max_iter : 
    print(f"Наближений розв'язок системи не обчислено за {cond} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Наближений розв'язок системи 
 [[1.00000002]
 [1.99999999]
 [3.        ]] 
 обчислено за 8 ітерацій


>#### Обчислення наближеного розв'язку СЛАР (1) методом Зейделя з точністю ``eps``

-------------------

In [56]:
cond, x = Seidel_solver(set_matrix, set_vector, set_x0, eps, max_iter)

if cond > 0 and cond < max_iter:
    print(f"Наближений розв'язок системи \n {x} \n обчислено за {cond} ітерацій")
elif  cond == max_iter : 
    print(f"Наближений розв'язок системи не обчислено за {cond} ітерацій")
else:
    print("Матриця СЛАР (1) не задовольняє достатню умову збіжності ітерацій")

Наближений розв'язок системи 
 [1.00000002 1.99999999 3.        ] 
 обчислено за 8 ітерацій


>#### Обчислення власних чисел і векторів матриці, отриманої для методу Якобі-Зейделя

-------------------

In [57]:
MinvL=set_matrix1()

In [58]:
la, v = linalg.eig(MinvL)

In [59]:
type(v)

numpy.ndarray

In [60]:
type(la[0])

numpy.complex128

In [61]:
l1, l2, l3 = la

print(l1, l2, l3)   # eigenvalues

0j (-0.018737548565432485+0j) (0.1067375485654325+0j)


In [62]:
print(np.abs(l1), np.abs(l2), np.abs(l3))

0.0 0.018737548565432485 0.1067375485654325


In [63]:
print(v[:, 0])   # first eigenvector
print(v[:, 1])   # second eigenvector
print(v[:, 2])   # third eigenvector

print(np.sum(abs(v**2), axis=0))  # eigenvectors are unitary

v1 = np.array(v[:, 0]).T
print(linalg.norm(MinvL.dot(v1) - l1*v1))

[1. 0. 0.]
[-0.96289452 -0.26642981  0.04300349]
[ 0.83507981 -0.51737697 -0.18698337]
[1. 1. 1.]
0.0
